## Predictions

Now we can generate predictions using the trained model. We'll first load the model and then specify stock symbols for prediction. The most recent data for the specified stock will be downloaded and fed foward through the trained network using a predictor class instance. The latter will reuse the code written earlier for loading datasets and will check model parameters to determine how much data (e.g., how many quarter's worth) need to be downloaded for a given model.

In [1]:
from model.classifier import StockClassifier
from model.predictor import StockClassifierPredictor

In [2]:
# Load model from disk
import torch
model = StockClassifier.from_file('data')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(model)

Last Train Date:     2020-03-10 10:30:15.416159
Train Count:         1
Prediction Interval: 90 days, 0:00:00
Data Interval:       360 days, 0:00:00
Trained Features: 
    Categorical: 5
    Daily:       2
    Quarterly:   6
Dimensions: 
    Embedded:    10
    Hidden:      500
    Output:      1


In [4]:
!pygmentize model/predictor.py

import os
import torch
import torch.utils.data
import numpy as np

from datetime import timedelta
from scipy.special import expit

from api.stocks import Stock   # Information about a particular stock
from api.stocks import TIMEDELTA_QUARTER
from api.stocks import TIMEDELTA_MONTH
from api.stocks import TIMEDELTA_YEAR

from model.dataset          import StockDataset
from model.predictor_helper import get_stock_cat_features_func
from model.predictor_helper import get_stock_daily_features_func
from model.predictor_helper import get_stock_quarterly_features_func

class StockClassifierPredictor():

    def __init__(self,model,device):
        """ 
        Args:
            model: The PyTorch model that we wish to use to make predictions.
            device: Where the model and data should be loaded (gpu or cpu).
        """
        self.model = model
        self.device = device

        self.get_stock_cat_features = get_stock_cat_features_func(self.model.data_interval,None)
        self.ge

In [5]:
# Create a predictor instance using our trained model.
predictor = StockClassifierPredictor(model,device)

In [6]:
# Let's get some predictions for next quarter ('XYZ' is a non-existent ticker, 
# and 'NLOK' is too new a company with insufficient data!)
for stock_ticker in ['AAPL','AMZN','BIIB','KSS','MSFT','NLOK','NFLX','T','TSLA','V','XYZ']:
    try:
        y = predictor.predict(stock_ticker)
        print(f'{stock_ticker+":": <6} {y}')
    except:
        print(f'{stock_ticker+":": <6} <Insufficient data or invalid ticker>')

AAPL:  1
AMZN:  1
BIIB:  0
KSS:   0
MSFT:  1
NLOK:  <Insufficient data or invalid ticker>
NFLX:  1
T:     0
TSLA:  0
V:     1
XYZ:   <Insufficient data or invalid ticker>
